## Correspondances (redondante)
tous les fonctions utilise dans ce notebook sont redonante. Car le resultat de ce notebook c'est le meme si on utilise une dataframe avec deux filtres :-(

In [46]:
import pandas as pd
import ipywidgets as widgets

In [47]:
airports = pd.read_csv("../data/aeroportsANLO.csv")
routes = pd.read_csv("../data/routesANLO.csv")

In [48]:
# creation du list from airports 
name_airports = airports['Name'].tolist()
iata_airports = airports['Iata'].tolist()

## Widgets 

In [49]:
airportDepart = widgets.Dropdown(
    options= name_airports,
    description='Airport Depart:',
    disabled=False,
)
airportDepart

Dropdown(description='Airport Depart:', options=('A Coruna Airport', 'Aachen Merzbruck Airport', 'Aalborg Airp…

In [50]:
airportArrive = widgets.Dropdown(
    options= name_airports,
    description='Airport Depart:',
    disabled=False,
)

airportArrive

Dropdown(description='Airport Depart:', options=('A Coruna Airport', 'Aachen Merzbruck Airport', 'Aalborg Airp…

In [51]:
nb_correspondances =widgets.BoundedIntText(
    value=1,
    min=0,
    max=1,
    step=1,
    description='Number of correpondances:',
    disabled=False
)

nb_correspondances

BoundedIntText(value=1, description='Number of correpondances:', max=1)

In [52]:
"On veut aller de {} à {} avec {} corresponances".format(airportDepart.value, airportArrive.value, nb_correspondances.value)

'On veut aller de A Coruna Airport à A Coruna Airport avec 1 corresponances'

## Fonctions

findAirportCode(..)

In [53]:
# d'abord on vais ecrire une function pour trouver une route entre deux airports donnees 
def findAirportCode(airportName : str):
    '''
    fonction pour trouver le airport code ('iata') si on a une airportname 
    retour : string with 3 letters corresponding to iata of airport 
    '''
    index = name_airports.index(airportName)
    return iata_airports[index]
        

findAirportCountry(...)

In [54]:
def findAirportCountry(airportIATA : str):
    '''
    fonction pour trouver le pays d'une airport si on l'IATA  
    retour : string corresponding to country of the airport 
    '''
    index = iata_airports.index(airportIATA)
    return airports['Country'][index]


#findAirportCountry('LIL') # 'France'

Connection_between_two_airports(.., ..)

In [55]:
def connection_between_two_airports(airportArriveCode : str, airportDepartCode : str):
    '''
    fonction qui verifie si on a une liaison/route entre les deux codes 
    retour : integer qui répresent le num_IATA ou une dict {0 : {airportC: ..., IATA : ....}}
    '''
    if airportDepartCode == '' or airportArriveCode == '':
      if airportDepartCode == '' : var1 = airportArriveCode
      else : var1 = airportDepartCode
      
      tmp1 = ["airportC", "IATA"]
      tmp = dict();
      index = 0 ; 
      for row in routes.iterrows():
        if(row[1][11] == var1):
          tmp2 = [row[1][12], row[1][13]]
          tmp3 = dict(zip(tmp1, tmp2))
          tmp[index] = tmp3 
          index = index + 1
      return tmp
    else: 
      for row in routes.iterrows():
        # 11 -> arrive, 12 --> depart, 13 --> num_IATA 
        if((row[1][11] == airportArriveCode) and (row[1][12] == airportDepartCode)):
          return row[1][13]

In [56]:
# fonction qui vérifie si une les deux codes de airports ont une liaison/route 
def connection_between_two_airports_v2(airportArriveCode : str, airportDepartCode : str):
    '''
    fonction qui verifie si on a une liaison/route entre les deux codes 
    retour : integer qui répresent le num_IATA ou un dict avec {airport : IATA}
    '''
    if airportDepartCode == '' or airportArriveCode == '':
      if airportDepartCode == '' : var1 = airportArriveCode
      else : var1 = airportDepartCode
      
      tmp = dict();
      for row in routes.iterrows():
        if(row[1][11] == var1):
          tmp[row[1][12]] = row[1][13] # dict avec comme key le code d'airport 
      return tmp
    
    else: 
      for row in routes.iterrows():
        # 11 -> arrive, 12 --> depart, 13 --> num_IATA 
        if((row[1][11] == airportArriveCode) and (row[1][12] == airportDepartCode)):
          return row[1][13]
    
#routes[routes['Depart'] == 'CDG'][routes['Arrivee'] == 'JFK']
#connection_between_two_airports_v2('CDG', '')    

In [57]:
#compare two dict to find the same airport 
def compare_dict_airport_v1(dict1 : dict, dict2 : dict):
    '''
    fonction qui retourne une list avec tous les airports que les deux dictionnaires ont en commen
    retour : list des airport codes 
    '''
    possibilites_duplicates = [] 
    ########## loop d1
    for k, content_k in dict1.items():
            ########### loop d2 
            for p, content_p in dict2.items():
                ## test if in those dict are the same airports 
                if content_k['airportC'] == content_p['airportC']:
                    possibilites_duplicates.append(content_k['airportC'])
    return list(set(possibilites_duplicates))
 

In [58]:
#connection_between_two_airports('JFK', '') # check 645 tuples 
#connection_between_two_airports('', 'JFK') # check 645 tuples 

# JFK -> LAX 
#d1 = connection_between_two_airports('JFK', '') # LAX
#d2 = connection_between_two_airports('', 'BOS') # LAX
#compare_dict_airport_v1(d1,d2) 
#print(connection_between_two_airports('JFK', 'BOS'))

remove_other_continents(... , ...)

In [70]:
continents_df = pd.read_csv("../data/countries.csv")
continents_dict = dict(zip(continents_df['country'], continents_df['continent']))
del continents_df

In [72]:
def remove_other_continents(list_airport_codes, continentStart):
    '''
    fonction qui enleve tous les airports_code qui ne sont pas sur le meme continent 
    retour : liste des airport_codes 
    '''
    country_matching_compare = []
    continent_matching_compare = []
    #finding the countries corresponding by the airports 
    for i in range(0, len(list_airport_codes)):
        country_matching_compare.append(findAirportCountry(list_airport_codes[i]))
    
    #finding the continent corresponding to the countries 
    for i in range(0, len(country_matching_compare)):
        
        continent_matching_compare.append(continents_dict[country_matching_compare[i]])
    
    combination = pd.DataFrame(); 
    combination['AirportCode'] = list_airport_codes
    combination['Continent'] = continent_matching_compare
    
    # renvoie seulement les airports qui sont sur le meme continent 
    return combination[combination['Continent'] == continentStart]['AirportCode'].to_list()

In [61]:
#old test
#dd1 = connection_between_two_airports_v2('JFK', '') # LAX 22.2
#dd1 = connection_between_two_airports_v2('BOS', '') # BOS 22.2


In [62]:
# old tests
#dd1 = connection_between_two_airports('JFK', '') # LAX 24.1
#dd1 = connection_between_two_airports('BOS', '') # LAX 24.1

In [63]:
def compare_dict_airport_v2(dict1 : dict, dict2 : dict):
    '''
    fonction qui retourne une list avec tous les airports que les deux dictionnaires ont en commen
    retour : list des airport codes 
    '''
    possibilites_duplicates = [] 
    ########## loop dict1
    for k, content_k in dict1.items():
            ########### loop d2 
            for p, content_p in dict2.items():
                ## test if in those dict are the same airports 
                if content_k['airportC'] == content_p['airportC']:
                    possibilites_duplicates.append(content_k['airportC'])
    return list(set(possibilites_duplicates))
  
#compare_dict_airport_v2(dd1, dd2)


In [64]:
def airports_same_continent(airportCode1 : str, airportCode2 : str):
    '''
    fonction qui verifie si les deux airports sont sur le meme continent, si cela c'est le cas le fonction enleve les airportsCode qui ne sont pas sur cette continent
    retour : list avec les airportCodes (str)
    '''
    c1 = findAirportCountry(airportCode1)
    c2 = findAirportCountry(airportCode2)
    
    con1 = continents_dict[c1]
    con2 = continents_dict[c2]
    
    if(con1 == con2):
        return con1
    else : return -1

liaison_correspondances(...) --> fonction qui est le but de ce notebook

bidect,,, index ,, snake vi,, icicle , line-by-line-profiling 

In [74]:
def liaison_correspondances(nb_correspondances : int, airportName1 : str, airportName2 : str): 
    '''
    fonction qui en fonction de nombre de correspondances donne les codes des airports qui pouvait être une correspondance 
    retour : soit un liste avec les code des airports, soit le code d'un route (depends de nb_correspondances)
    '''
    c1 = findAirportCode(airportName1)
    c2 = findAirportCode(airportName2)

    if nb_correspondances == 0 :
        return connection_between_two_airports(c1, c2)
    elif nb_correspondances == 1 :
        d1 = connection_between_two_airports(c1, '')
        d2 = connection_between_two_airports('', c2)
        
        r = airports_same_continent(c1, c2)
        if r != -1:
            return remove_other_continents(compare_dict_airport_v1(d1, d2), r)
        
print(liaison_correspondances(1, 'Aalborg Airport', 'Amsterdam Schiphol Airport'))

#[df_vols[df_vols['Depart'] == 'aal]['Arrivee']]
#[df_vols[df_vols['Depart'] == 'ams']['Arrivee']]
#pd.erge( d1 ,d2 , on='arrivee, how= 'inner)

#print(liaison_correspondances(1, 'New York John F. Kennedy International Airport', 'Paris Charles de Gaulle Airport'))

['MPL', 'CPH', 'DUS', 'BLL', 'AGP', 'OTP', 'STN', 'LPA', 'TFS', 'FUE', 'OSL']
